# MOUNT DRIVE & PATH

In [1]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/bigdata_final_project"
WAREHOUSE_PATH = f"{PROJECT_ROOT}/warehouse"
DB_PATH = f"{WAREHOUSE_PATH}/bigdata_warehouse.db"


Mounted at /content/drive


In [2]:
import os
print(os.path.exists(DB_PATH))


True


# IMPORT & CONNECT SQLITE

In [6]:
import sqlite3
import pandas as pd
from datetime import datetime
import logging
import os
from sqlalchemy import create_engine, text
from google.colab import drive

conn = sqlite3.connect(DB_PATH)


# TRANSFORM WEATHER

## CEK INPUT WEATHER RAW

In [4]:
pd.read_sql(
    "SELECT * FROM raw_weather LIMIT 5;",
    conn
)


,index,time,temperature_2m,rain,snowfall
0,0,2025-01-01 00:00:00,9.1,0.9,0.0
1,1,2025-01-01 01:00:00,8.6,0.0,0.0
2,2,2025-01-01 02:00:00,8.9,0.0,0.0
3,3,2025-01-01 03:00:00,8.8,0.0,0.0
4,4,2025-01-01 04:00:00,8.3,0.0,0.0


## TRANSFORM WEATHER (SQL)

Tambah kolom jam (hour)

Handle hujan (is_rainy)

Tandai jam sibuk (is_rush_hour)

Tetap simpan kolom numerik utama

Semua dilakukan di SQL (ELT)

In [17]:
db_engine = create_engine(f'sqlite:///{DB_PATH}')
connection = db_engine.connect()

logging.info("Reset tabel tujuan 'fact_trips_elt'...")
connection.execute(text("DROP TABLE IF EXISTS fact_trips_elt"))

# Query Utama Transformasi ELT (SUDAH DIPERBAIKI NAMA KOLOMNYA)
# pickup_datetime      -> Waktu jemput
# tpep_dropoff_datetime -> Waktu turun

query_transform = """
CREATE TABLE fact_trips_elt AS
SELECT
    ROW_NUMBER() OVER (ORDER BY t.pickup_datetime) as trip_id,
    t.pickup_datetime,
    t.tpep_dropoff_datetime,
    t.passenger_count,
    t.trip_distance,
    t.PULocationID,
    t.DOLocationID,
    t.total_amount,
    w.temperature_2m as temperature,
    w.rain as rainfall_mm,

    -- FITUR 1: Durasi (Menit)
    ROUND((JULIANDAY(t.tpep_dropoff_datetime) - JULIANDAY(t.pickup_datetime)) * 1440, 2) as trip_duration_minutes,

    -- FITUR 2: Kecepatan Rata-rata (MPH)
    CASE
        WHEN (JULIANDAY(t.tpep_dropoff_datetime) - JULIANDAY(t.pickup_datetime)) * 24 = 0 THEN 0
        ELSE ROUND(t.trip_distance / ((JULIANDAY(t.tpep_dropoff_datetime) - JULIANDAY(t.pickup_datetime)) * 24), 2)
    END as average_speed_mph,

    -- FITUR 3: Waktu (Pagi/Siang/Malam)
    CASE
        WHEN CAST(STRFTIME('%H', t.pickup_datetime) AS INT) BETWEEN 6 AND 11 THEN 'Pagi'
        WHEN CAST(STRFTIME('%H', t.pickup_datetime) AS INT) BETWEEN 12 AND 17 THEN 'Siang'
        WHEN CAST(STRFTIME('%H', t.pickup_datetime) AS INT) BETWEEN 18 AND 23 THEN 'Malam'
        ELSE 'Dini Hari'
    END as time_of_day,

    -- FITUR 4: Is Weekend
    CASE
        WHEN STRFTIME('%w', t.pickup_datetime) IN ('0', '6') THEN 1
        ELSE 0
    END as is_weekend,

    -- FITUR 5: Cost per Mile
    CASE
        WHEN t.trip_distance = 0 THEN 0
        ELSE ROUND(t.total_amount / t.trip_distance, 2)
    END as cost_per_mile

FROM raw_taxi t
LEFT JOIN raw_weather w
    ON STRFTIME('%Y-%m-%dT%H:00', t.pickup_datetime) = w.time
WHERE t.trip_distance > 0 AND t.total_amount > 0
"""
logging.info("Menjalankan Query Transformasi...")
connection.execute(text(query_transform))

result = connection.execute(text("SELECT count(*) FROM fact_trips_elt")).scalar()
logging.info(f"Selesai! Tabel 'fact_trips_elt' berisi {result} baris.")
logging.info(f"Lokasi DB: {DB_PATH}")



## VERIFIKASI HASIL WEATHER CLEAN

In [30]:
pd.read_sql(
    "SELECT * FROM fact_trips_elt LIMIT 10;",
    conn
)


,trip_id,pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount,temperature,rainfall_mm,trip_duration_minutes,average_speed_mph,time_of_day,is_weekend,cost_per_mile
0,1,2025-01-01 00:00:31,2025-01-01 00:04:35,1.0,0.54,234,164,15.80,None,None,4.07,7.97,Dini Hari,0,29.26
1,2,2025-01-01 00:00:37,2025-01-01 00:04:46,1.0,0.86,140,263,11.50,None,None,4.15,12.43,Dini Hari,0,13.37
2,3,2025-01-01 00:01:00,2025-01-01 00:07:39,1.0,3.63,238,244,21.72,None,None,6.65,32.75,Dini Hari,0,5.98
3,4,2025-01-01 00:01:20,2025-01-01 00:24:16,1.0,9.07,132,62,43.25,None,None,22.93,23.73,Dini Hari,0,4.77
4,5,2025-01-01 00:01:44,2025-01-01 00:50:35,1.0,19.30,132,239,94.65,None,None,48.85,23.71,Dini Hari,0,4.90
5,6,2025-01-01 00:01:44,2025-01-01 00:08:38,0.0,0.90,113,148,13.90,None,None,6.90,7.83,Dini Hari,0,15.44
6,7,2025-01-01 00:02:20,2025-01-01 00:03:34,1.0,0.12,249,249,8.70,None,None,1.23,5.84,Dini Hari,0,72.50
7,8,2025-01-01 00:03:58,2025-01-01 00:41:28,1.0,27.74,132,265,112.19,None,None,37.50,44.38,Dini Hari,0,4.04
8,9,2025-01-01 00:03:58,2025-01-01 00:24:40,1.0,5.75,249,238,40.62,None,None,20.70,16.67,Dini Hari,0,7.06
9,10,2025-01-01 00:04:29,2025-01-01 00:35:21,4.0,2.01,163,164,36.48,None,None,30.87,3.91,Dini Hari,0,18.15


In [ ]:
result = connection.execute(text("SELECT count(*) FROM fact_trips_elt")).scalar()
logging.info(f"Selesai! Tabel 'fact_trips_elt' berhasil dibuat dengan {result} baris.")

CEK STRUKTUR TABEL

In [20]:
pd.read_sql(
    "PRAGMA table_info(fact_trips_elt);",
    conn
)


,cid,name,type,notnull,dflt_value,pk
0,0,trip_id,,0,None,0
1,1,pickup_datetime,TEXT,0,None,0
2,2,tpep_dropoff_datetime,TEXT,0,None,0
3,3,passenger_count,REAL,0,None,0
4,4,trip_distance,REAL,0,None,0
5,5,PULocationID,INT,0,None,0
6,6,DOLocationID,INT,0,None,0
7,7,total_amount,REAL,0,None,0
8,8,temperature,REAL,0,None,0
9,9,rainfall_mm,REAL,0,None,0


# **PENGARUH CUACA TERHADAP TRAFFIC**

In [29]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import drive

def analyze_weather_impact_elt():
    print("=== ANALISIS PENGARUH CUACA (DATA ELT) ===")


    # Koneksi ke SQLite
    conn = sqlite3.connect(DB_PATH)

    # 2. Ambil Data dari Tabel ELT
    # Kita mengambil kolom waktu, uang, durasi, dan cuaca
    query = """
    SELECT
        pickup_datetime,
        total_amount,
        trip_duration_minutes,
        temperature,
        rainfall_mm
    FROM fact_trips_elt
    """

    try:
        df = pd.read_sql(query, conn)

        # Konversi ke Datetime
        df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

        # 3. Feature Engineering Sederhana untuk Visualisasi
        # Kategori Cuaca: Hujan jika curah hujan > 0.1 mm
        df['weather_condition'] = df['rainfall_mm'].apply(
            lambda x: 'Hujan (>0.1mm)' if x > 0.1 else 'Cerah/Berawan'
        )

        # Agregasi Data per Jam (untuk melihat Traffic/Volume order)
        df['date_hour'] = df['pickup_datetime'].dt.floor('H')

        hourly_stats = df.groupby(['date_hour', 'weather_condition']).agg({
            'pickup_datetime': 'count',      # Jumlah Order (Traffic)
            'total_amount': 'sum',           # Total Omzet
            'trip_duration_minutes': 'mean', # Rata-rata Durasi
            'rainfall_mm': 'mean',
            'temperature': 'mean'
        }).rename(columns={'pickup_datetime': 'total_orders'}).reset_index()

        # ==========================================
        # 4. VISUALISASI
        # ==========================================
        sns.set(style="whitegrid")
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # GRAFIK 1: Perbandingan Volume Order (Traffic) Hujan vs Cerah
        # Kita pakai rata-rata order per jam
        sns.barplot(x='weather_condition', y='total_orders', data=hourly_stats, ax=axes[0, 0], palette='viridis', estimator='mean')
        axes[0, 0].set_title('Rata-rata Order Taksi per Jam (Traffic)')
        axes[0, 0].set_ylabel('Jumlah Order Rata-rata')
        axes[0, 0].set_xlabel('Kondisi Cuaca')

        # GRAFIK 2: Hubungan Curah Hujan vs Durasi Perjalanan
        # Apakah hujan bikin macet (durasi naik)?
        sns.scatterplot(x='rainfall_mm', y='trip_duration_minutes', data=hourly_stats, ax=axes[0, 1], color='orange', alpha=0.6)
        axes[0, 1].set_title('Curah Hujan vs Rata-rata Durasi Perjalanan')
        axes[0, 1].set_xlabel('Curah Hujan (mm)')
        axes[0, 1].set_ylabel('Durasi (Menit)')

        # GRAFIK 3: Distribusi Total Pendapatan per Jam
        sns.boxplot(x='weather_condition', y='total_amount', data=hourly_stats, ax=axes[1, 0], palette='coolwarm')
        axes[1, 0].set_title('Distribusi Pendapatan Taksi per Jam')
        axes[1, 0].set_ylabel('Total Pendapatan ($)')

        # GRAFIK 4: Matriks Korelasi
        # Melihat angka hubungan antar variabel
        corr_cols = ['total_orders', 'total_amount', 'trip_duration_minutes', 'rainfall_mm', 'temperature']
        corr_matrix = hourly_stats[corr_cols].corr()
        sns.heatmap(corr_matrix, annot=True, cmap='RdBu', center=0, ax=axes[1, 1], fmt=".2f")
        axes[1, 1].set_title('Korelasi: Cuaca vs Bisnis Taksi')

        plt.tight_layout()
        plt.show()

        # Print Insight Teks Singkat
        print("\n=== KESIMPULAN DATA ===")
        avg_rain_traffic = hourly_stats[hourly_stats['weather_condition'] == 'Hujan (>0.1mm)']['total_orders'].mean()
        avg_dry_traffic = hourly_stats[hourly_stats['weather_condition'] == 'Cerah/Berawan']['total_orders'].mean()

        print(f"Rata-rata Order/Jam saat HUJAN: {avg_rain_traffic:.1f}")
        print(f"Rata-rata Order/Jam saat CERAH: {avg_dry_traffic:.1f}")

        if avg_rain_traffic > avg_dry_traffic:
            print("-> INSIGHT: Permintaan taksi cenderung NAIK saat hujan.")
        else:
            print("-> INSIGHT: Permintaan taksi cenderung TURUN/SAMA saat hujan.")

    except Exception as e:
        print(f"Error Visualisasi: {e}")
    finally:
        conn.close()

# Jalankan
analyze_weather_impact_elt()

=== ANALISIS PENGARUH CUACA (DATA ELT) ===
Error Visualisasi: Execution failed on sql '
    SELECT 
        pickup_datetime, 
        total_amount, 
        trip_duration_minutes,
        temperature, 
        rainfall_mm 
    FROM fact_trips_elt
    ': no such table: fact_trips_elt
